<a href="https://colab.research.google.com/github/jvishnuvardhan/Medium_Articles/blob/master/Saving_Loading_with_CustomLossLayer_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.2

In [2]:
# Import modules
import tensorflow as tf
from tensorflow import keras
import os, datetime

Epochs = 2
# Load data
mnist = tf.keras.datasets.mnist

# Split the data to train and test
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# Process the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define model
def create_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='Flatten'),
    tf.keras.layers.Dense(512, activation='relu', name = 'Dense1'),
    tf.keras.layers.Dropout(0.2, name = 'Dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name = 'Output')
    ])
  
  return model

# Custom Loss Layer
class MyLoss_layer(keras.losses.Loss):
  def __init__(self, from_logits=False,
                reduction=keras.losses.Reduction.AUTO,
                name='MyLoss_layer'):
      super(MyLoss_layer, self).__init__(reduction=reduction, name=name)

  def call(self, y_true, y_pred):
    custom_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return custom_loss

# Create a basic model instance
model = create_model()

# Compile the model
model.compile(optimizer='adam',
            loss=MyLoss_layer(),
            metrics=['sparse_categorical_accuracy']) 

# define history callback (you can use it to access some parameters under it)
history = model.fit(x_train, y_train, epochs = Epochs, verbose=1,batch_size=128)

loss, acc  = model.evaluate(x_test, y_test, verbose=10) 
print("Original model, loss:{:5.2f} and accuracy: {:5.2f}% ".format(loss, 100*acc))

Epoch 1/2
469/469 [==============================] - 5s 10ms/step - loss: 0.2842 - sparse_categorical_accuracy: 0.9194
Epoch 2/2
469/469 [==============================] - 4s 9ms/step - loss: 0.1224 - sparse_categorical_accuracy: 0.9649
Original model, loss: 0.10 and accuracy: 96.95% 


In [3]:
# Saving entire model in tf format
model.save("MyModel_tf",save_format='tf')

# Saving entire model in h5 format
model.save("MyModel_h5",save_format='h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: MyModel_tf/assets


In [5]:
# Loading the saved model and testing the performance (tf format)
loaded_model_tf = tf.keras.models.load_model('MyModel_tf',custom_objects={'MyLoss_layer':MyLoss_layer()}) # 

# Evaluate the loaded model after loading weights from tf 
loss, acc_tf = loaded_model_tf.evaluate(x_test, y_test, verbose=1)
print("Loaded model (tf), loss:{:5.2f} and accuracy: {:5.2f}% ".format(loss, 100*acc_tf))

313/313 [==============================] - 1s 2ms/step - loss: 0.1031 - sparse_categorical_accuracy: 0.9695
Loaded model (tf), loss: 0.10 and accuracy: 96.95% 


In [6]:
# Loading the saved model and testing the performance (h5 format)
loaded_model_h5 = tf.keras.models.load_model('MyModel_h5', custom_objects={'MyLoss_layer':MyLoss_layer()})


# Evaluate the loaded model after loading weights from h5 
loss, acc_h5 = loaded_model_h5.evaluate(x_test, y_test, verbose=1)
print("Loaded model (h5), loss:{:5.2f} and accuracy: {:5.2f}% ".format(loss, 100*acc_h5))

313/313 [==============================] - 1s 2ms/step - loss: 0.1031 - sparse_categorical_accuracy: 0.9695
Loaded model (h5), loss: 0.10 and accuracy: 96.95% 
